In [1]:
import os
from collections import Counter
from keras.preprocessing.text import Tokenizer
import numpy as np
from bs4 import BeautifulSoup
import re
import xlrd

# 1 - Preprocessing

In [2]:
input_path = "NICT_JLE_4.1/LearnerOriginal"
xls_path = "NICT_JLE_4.1/NICT_JLE_list.xls"

#PARAMETERS
output_path = "preprocessed_text/"

train_data_path = "train_data/"
test_data_path = "test_data/"

ratio_train_test = 0.7

In [3]:
if not os.path.exists(output_path+train_data_path):
	os.makedirs(output_path+train_data_path)

if not os.path.exists(output_path+test_data_path):
	os.makedirs(output_path+test_data_path)


wb = xlrd.open_workbook(xls_path)
sheet = wb.sheet_by_index(0)

os.chdir(input_path)

folder_len = len([name for name in os.listdir() if (os.path.isfile(name) and name.endswith(".txt"))])

cnt = 1

for input_file in os.listdir():
    if input_file.endswith(".txt"):

        input_text = open(input_file,'r', errors='ignore')
        input_string = input_text.read()
        input_text.close()

        soup = BeautifulSoup(input_string,'lxml')
        input_soup = soup.find_all('b')
        input_string = "".join(str(input_soup))

        soup = BeautifulSoup(input_string,'lxml')
        input_soup = soup.get_text()
        input_string = "".join(str(input_soup))

        input_string = re.sub(">","",input_string)

        pattern = re.compile('[a-zA-Z_ ]')
        matches = pattern.findall(input_string)
        preproccesed_text = "".join(matches)

        if cnt < folder_len*ratio_train_test :
        	os.chdir("../../"+output_path+train_data_path)
        else :
        	os.chdir("../../"+output_path+test_data_path)

        for row_num in range(sheet.nrows):
            row_value = sheet.row_values(row_num)
            if row_value[0].split(".")[0] == input_file.split(".")[0] :
                output_text = open(input_file.split(".")[0]+"_"+str(int(row_value[2]))+".txt", "w") 
                output_text.write(preproccesed_text)
                output_text.close() 

        print("Preprocessing = "+str(cnt)+"/"+str(folder_len), end="\r")

        cnt += 1
        os.chdir("../../"+input_path)

# 2 - Bag of words

In [4]:
train_data_path = "preprocessed_text/train_data/"
test_data_path = "preprocessed_text/test_data/"

output_train_data_path = "preprocessed_text/matrix_train_data/"
output_test_data_path = "preprocessed_text/matrix_test_data/"

#PARAMETERS
MIN_OCCURANE = 2

### VOCABULARY

In [5]:
def create_vocab(path, vocab):

    os.chdir(path)

    for input_file in os.listdir():
        if input_file.endswith(".txt"):
            input_text = open(input_file,'r')
            input_string = input_text.read()
            input_text.close()

            tokens = input_string.split()
            vocab.update(tokens)

    tokens = [k for k,c in vocab.items() if c >= MIN_OCCURANE]

    os.chdir("../../")

    vocab_string = ' '.join(tokens)
    output_file = open("vocab.txt",'w')
    output_file.write(vocab_string)
    output_file.close()

    return tokens, vocab

### DATASET

In [6]:
def transform_dataset(path, tokens, tokenizer) :
    docs = list()

    os.chdir(path)

    for input_file in os.listdir():
        if input_file.endswith(".txt"):
            input_text = open(input_file,'r')
            input_string = input_text.read()
            input_text.close()

            tokens_dataset = input_string.split()
            tokens_dataset = [w for w in tokens_dataset if w in tokens]
            line = ' '.join(tokens_dataset)
            docs.append(line)

    if path.endswith("train_data/"):
        tokenizer.fit_on_texts(docs)

    tokenized_data = tokenizer.texts_to_matrix(docs, mode='freq')

    os.chdir("../../")

    return tokenizer, tokenized_data

### OUTPUT

In [7]:
def output_matrix(input_path, output_path, tokenized_data):
    name_list = list()

    os.chdir(input_path)

    for input_file in os.listdir():
        if input_file.endswith(".txt"):
            name_list.append(input_file.split(".")[0])


    if not os.path.exists("../../"+output_path):
        os.makedirs("../../"+output_path)
    os.chdir("../../"+output_path)

    i = 0
    for name in name_list :
        np.savetxt(name+'.out', tokenized_data[i], delimiter=',')
        i += 1

    os.chdir("../../")

### PROGRAM

In [8]:
vocab = Counter()
tokens = []

tokenizer = Tokenizer()

tokens, vocab = create_vocab(train_data_path,vocab)
tokens, vocab = create_vocab(test_data_path,vocab)

print("The 50 most common words: "+str(vocab.most_common(50)))

tokenizer, tokenized_train_data = transform_dataset(train_data_path,tokens,tokenizer)
tokenizer, tokenized_test_data = transform_dataset(test_data_path,tokens,tokenizer)

print("tokenized_train_data.shape : "+str(tokenized_train_data.shape))
print("tokenized_test_data.shape : "+str(tokenized_test_data.shape))

output_matrix(train_data_path, output_train_data_path, tokenized_train_data)
output_matrix(test_data_path, output_test_data_path, tokenized_test_data)

The 50 most common words: []


ValueError: Specify a dimension (`num_words` argument), or fit on some text data first.